# Chapter 7

## Problem 1

Find eigenvalues and eigenvectors of the matrix:

\begin{equation}
A = \begin{pmatrix} 10 & 7 & 8 & 7\\ 7 & 5 & 6 & 5\\ 8 & 6 & 10 & 9\\ 7 & 5 & 9 & 10 \end{pmatrix}
\end{equation}

Lets put the matrix in the Jacobi code from lecture notes.

In [38]:
using DataStructures
import DataStructures: PriorityQueue, peek

function jacobi_rotation(a::T, b::T, c::T) where T <: Real
    ratio = (c - a) / (2b)
    if ratio >= zero(T)
        tangent = one(T) / (ratio + sqrt(one(T) + ratio^2))
    else
        tangent = -one(T) / (abs(ratio) + sqrt(one(T) + ratio^2))
    end
    cosine = one(T) / sqrt(one(T) + tangent^2)
    sine = tangent * cosine
    return (cosine, sine)
end

function jacobi(M::Matrix{T}, tol::T) where T <: Real
    n = size(M, 1)
    m = div(n * (n-1), 2)
    key = Vector{Tuple{Int, Int}}(m) # set up keys and priorities
    priority = Vector{Float64}(m)
    k = 0
    for i = 1:n
        for j = (i + 1):n
            k = k +1
            key[k] = (i, j)
            priority[k] = -abs(M[i, j])
        end
    end
    pq = PriorityQueue(key, priority) # construct queue
    #print(pq)
    eigenvector = eye(T, n)
    (i, j), val = peek(pq) # choose biggest off-diagonal entry
    while abs(val) > tol # perform Jacobi rotations until convergence
        (cosine, sine) = jacobi_rotation(M[i, i], M[i, j], M[j, j])
        u = M[i, :]
        v = M[j, :]
        M[i, :] = cosine * u - sine * v
        M[j, :] = sine * u + cosine * v
        u = M[:, i]
                v = M[:, j]
        M[:, i] = cosine * u - sine * v
        M[:, j] = sine * u + cosine * v
        u = eigenvector[:, i]
        v = eigenvector[:, j]
        eigenvector[:, i] = cosine * u - sine * v
        eigenvector[:, j] = sine * u + cosine * v
        for k = 1:n # update priority queue
            if k > i
                pq[(i, k)] = -abs(M[i, k])
            elseif k < i
                pq[(k, i)] = -abs(M[k, i])
            end
            if k > j
                pq[(j, k)] = -abs(M[j, k])
            elseif k < j
                pq[(k, j)] = -abs(M[k, j])
            end
        end
        (i, j), val = peek(pq) # choose next off-diagonal entry
    end
    return (diag(M), eigenvector)
end

jacobi (generic function with 1 method)

In [39]:
M = [10.0 7.0 8.0 7.0; 7.0 5.0 6.0 5.0; 8.0 6.0 10.0 9.0; 7.0 5.0 9.0 10.0];
#M = [2.0 1.0; 1.0 1.0]
Msave = copy(M);
tol = 1e-12;
(eigenvalue, eigenvector) = jacobi(M, tol);
vecnorm(Msave * eigenvector - eigenvector * diagm(eigenvalue))
print(eigenvalue)
print(eigenvector)

[3.85806, 0.01015, 0.843107, 30.2887][0.614861 -0.501565 -0.301652 0.528568; 0.396306 0.830444 0.093305 0.380262; -0.271601 -0.208554 0.760318 0.551955; -0.625396 0.123697 -0.567641 0.520925]

The Characteristic Polynomial for the matrix is:
  $x^4 - 35x^3 + 146x^2 - 100x + 1$

The eigenvalues are all real and are:   
${ 3.85806, 0.01015, 0.843107, 30.2887 }$

Eigenvectors:

 for Eigenvalue 0.01015: $[0.396306, 0.830444, 0.093305, 0.380262]$

 for Eigenvalue 0.843107: $[-0.271601, -0.208554, 0.760318, 0.551955]$

 for Eigenvalue 3.85806: $[0.614861, -0.501565, -0.301652, 0.528568]$

 for Eigenvalue 30.2887: $[-0.625396, 0.123697, -0.567641, 0.520925]$

## Problem 2

Find eigenvalues and eigenvectors of the rotation matrix:

\begin{equation}
A = \begin{pmatrix} cos \theta & -sin \theta\\ sin \theta & cos \theta \end{pmatrix}
\end{equation}


$Ax = \lambda x$

$ \implies (A - \lambda I)x = 0$

\begin{equation}
\implies
(A - \lambda I)x = 
\begin{pmatrix} (\cos \theta - \lambda)& -\sin \theta\\ \sin \theta & (\cos \theta - \lambda) \end{pmatrix}x
= 0
\end{equation}

The Characteristic Polynomial for the matrix is the value of the determinant $(A - \lambda I)$: 

$(\cos \theta - \lambda)^2 - (-\sin \theta)(\sin \theta) = 0$

$ \implies \cos^2 \theta + \lambda^2 - 2\lambda \cos \theta + \sin^2 \theta = 0$

$ \implies 1 + \lambda^2 - 2\lambda \cos \theta = 0$

$ \implies \lambda = \frac{2 \cos \theta \pm \sqrt{4 \cos^2 \theta - 4}}{2}$

$ \implies \lambda = \cos \theta \pm i\sin\theta$

Hence, the eigenvalues are $e^{i\theta}$ and $e^{-i\theta}$.

The eigenvalues are complex conjugates of each other and the eigenvector will also be in Complex space. 

To calculate eigenvector $(ae^{i\alpha}, be^{i\beta})^T$ corresponding to $\lambda = e^{i\theta}$:

\begin{equation}
\implies
(A - I)x = 
\begin{pmatrix} (\cos \theta - e^{i\theta})& -\sin \theta\\ \sin \theta & (\cos \theta - e^{i\theta}) \end{pmatrix}
\begin{pmatrix} ae^{i\alpha} \\ be^{i\beta} \end{pmatrix}
= 0
\end{equation}

\begin{equation}
\implies
\begin{pmatrix} ae^{i\alpha}(-i\sin \theta) -  be^{i\beta} \sin \theta  \\ ae^{i\alpha}\sin \theta  + be^{i\beta}(-i\sin \theta) \end{pmatrix}
= \begin{pmatrix} 0 \\ 0 \end{pmatrix}
\end{equation}

\begin{equation}
\implies
be^{i\beta} = ae^{i\alpha}(-i)
\end{equation}

So, $(1, -i)$ is the eigenvector corresponding to eigenvalue $e^{i\theta}$, which can be renormalized to $(\frac{e^{i0}}{\sqrt{2}}, \frac{e^{i\frac{-\pi}{2}}}{\sqrt{2}})$.

To calculate eigenvector $(ae^{i\alpha}, be^{i\beta})^T$ corresponding to $\lambda = e^{-i\theta}$:

\begin{equation}
\implies
(A - I)x = 
\begin{pmatrix} (\cos \theta - e^{-i\theta})& -\sin \theta\\ \sin \theta & (\cos \theta - e^{-i\theta}) \end{pmatrix}
\begin{pmatrix} ae^{i\alpha} \\ be^{i\beta} \end{pmatrix}
= 0
\end{equation}

\begin{equation}
\implies
\begin{pmatrix} ae^{i\alpha}(i\sin \theta) -  be^{i\beta} \sin \theta  \\ ae^{i\alpha}\sin \theta  + be^{i\beta}(i\sin \theta) \end{pmatrix}
= \begin{pmatrix} 0 \\ 0 \end{pmatrix}
\end{equation}

\begin{equation}
\implies
be^{i\beta} = ae^{i\alpha}(i)
\end{equation}

So, $(1, i)$ is the eigenvector corresponding to eigenvalue $e^{-i\theta}$, which can be renormalized to $(\frac{e^{i0}}{\sqrt{2}}, \frac{e^{i\frac{\pi}{2}}}{\sqrt{2}})$.

Eigenvectors:

 for Eigenvalue $e^{i\theta}$: $(\frac{e^{i0}}{\sqrt{2}}, \frac{e^{i\frac{-\pi}{2}}}{\sqrt{2}})$

 for Eigenvalue $e^{-i\theta}$: $(\frac{e^{i0}}{\sqrt{2}}, \frac{e^{i\frac{\pi}{2}}}{\sqrt{2}})$

## Problem 3

Find eigenvalues and eigenvectors of the reflection matrix:

\begin{equation}
A = \begin{pmatrix} cos \theta & sin \theta\\ sin \theta & -cos \theta \end{pmatrix}
\end{equation}


$Ax = \lambda x$

$ \implies (A - \lambda I)x = 0$

\begin{equation}
\implies
(A - \lambda I)x = 
\begin{pmatrix} (\cos \theta - \lambda)& \sin \theta\\ \sin \theta & (-\cos \theta - \lambda) \end{pmatrix}x
= 0
\end{equation}

The Characteristic Polynomial for the matrix is the value of the determinant $(A - \lambda I)$: 

$(\cos \theta - \lambda)(-\cos \theta - \lambda) - (\sin \theta)(\sin \theta) = 0$

$\implies \lambda^2 - \cos^2 \theta - \sin^2 \theta = 0$

$ \implies \lambda^2 - 1 = 0$

$ \implies \lambda = \pm 1$

Hence, the eigenvalues are $1$ and $-1$.

To calculate eigenvector $(x_1, x_2)^T$ corresponding to $\lambda = 1$:

\begin{equation}
\implies
(A - I)x = 
\begin{pmatrix} (\cos \theta - 1)& \sin \theta\\ \sin \theta & (-\cos \theta - 1) \end{pmatrix}
\begin{pmatrix} x_1 \\ x_2 \end{pmatrix}
= 0
\end{equation}

\begin{equation}
\implies
\begin{pmatrix} x_1(\cos \theta - 1) +  x_2 \sin \theta  \\ x_1\sin \theta  + x_2(-\cos \theta - 1) \end{pmatrix}
= \begin{pmatrix} 0 \\ 0 \end{pmatrix}
\end{equation}

\begin{equation}
\implies
x_2 = \frac {x_1 (1 - \cos \theta)}{\sin \theta} = \frac {x_1 (2 \sin^2 \frac{\theta}{2})}{2 \sin \frac{\theta}{2} \cos \frac{\theta}{2}} = \frac {x_1 (\sin \frac{\theta}{2})}{\cos \frac{\theta}{2}} = x_1 \tan \frac{\theta}{2}
\end{equation}

So, $(1, \tan \frac{\theta}{2})$ is the eigenvector corresponding to eigenvalue 1, which can be renormalized to $(\cos \frac{\theta}{2}, \sin \frac{\theta}{2})$.

To calculate eigenvector $(x_1, x_2)^T$  corresponding to $\lambda = -1$:

\begin{equation}
\implies
(A + I)x = 
\begin{pmatrix} (\cos \theta + 1)& \sin \theta\\ \sin \theta & (-\cos \theta + 1) \end{pmatrix}
\begin{pmatrix} x_1 \\ x_2 \end{pmatrix}
= 0
\end{equation}

\begin{equation}
\implies
\begin{pmatrix} x_1(1 + \cos \theta) +  x_2 \sin \theta  \\ x_1\sin \theta  + x_2(1 -\cos \theta) \end{pmatrix}
= \begin{pmatrix} 0 \\ 0 \end{pmatrix}
\end{equation}

\begin{equation}
\implies
x_2 =  - \frac {x_1 (1 + \cos \theta)}{\sin \theta} = - \frac {x_1 (2 \cos^2 \frac{\theta}{2})}{2 \sin \frac{\theta}{2} \cos \frac{\theta}{2}} = - \frac {x_1 (\cos \frac{\theta}{2})}{\sin \frac{\theta}{2}} = - x_1 \cot \frac{\theta}{2}
\end{equation}

So, $(1, - \cot \frac{\theta}{2})$ is the eigenvector corresponding to eigenvalue -1, which can be renormalized to $(\sin \frac{\theta}{2}, - \cos \frac{\theta}{2})$.

Eigenvectors:

 for Eigenvalue 1: $(\cos \frac{\theta}{2}, \sin \frac{\theta}{2})$

 for Eigenvalue -1: $(\sin \frac{\theta}{2}, - \cos \frac{\theta}{2})$